In [3]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

from segmentation.util.utils_pipeline import segment_video_using_config_2d
from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import numpy as np
import napari
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import get_single_volume

In [5]:
# project_fname = r'Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\project_config.yaml'
project_fname = r'Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-2021_05_11\project_config.yaml'
project_cfg = load_config(project_fname)
segment_fname = project_cfg['subfolder_configs']['segmentation']
project_dir = Path(project_fname).parent
with safe_cd(project_dir):
    segment_cfg = load_config(segment_fname)

this_config = segment_cfg.copy()
this_config['dataset_params'] = project_cfg['dataset_params'].copy()

red_data_fname = project_cfg['red_bigtiff_fname']

In [6]:
with safe_cd(project_dir):
    fname = segment_cfg['output']['masks']
    z = zarr.open(fname)

In [7]:
viewer = napari.view_labels(z)

## View original

In [13]:
mask0_raw = np.array(z[0,...])
dat0_raw = get_single_volume(red_data_fname, 0, 33, dtype='uint16')

In [49]:
viewer = napari.view_labels(mask0_raw)
viewer.add_image(dat0_raw)

<Image layer 'dat0_raw' at 0x21ca6b7c688>

## Remove globally dim pixels and re-stitch

In [9]:
from segmentation.util.utils_postprocessing import *

In [46]:
background = 2 * np.mean(dat0_raw)
print(f"Background measured as: {background}")
dim_pixels = dat0_raw < background

mask0_dim_removed = mask0_raw.copy()
mask0_dim_removed[dim_pixels] = 0

Background measured as: 233.30953810953812


In [91]:
viewer = napari.view_labels(mask0_raw)
viewer.add_labels(mask0_dim_removed)
# viewer.add_image(dat0_raw)

<Labels layer 'mask0_dim_removed' at 0x21ca9300588>

## 3d stardist

In [81]:
from segmentation.util.utils_model import segment_with_stardist_3d
from segmentation.util.utils_model import get_stardist_model
from csbdeep.utils import Path, normalize

model_name = 'charlie_3d'
sd_model = get_stardist_model(model_name, verbose=2)

Getting Stardist model: charlie_3d
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.353429, nms_thresh=0.3.


In [82]:
def _segment3d(dat, model, prob_thresh=0.5):
    # From: segment_with_stardist_3d
    
    # initialize variables
    axis_norm = (0, 1, 2)
    n_channel = 1

    # normalizing images (stardist function)
    img = normalize(dat, 1, 99.8, axis=axis_norm)

    # run the prediction
    labels, details = model.predict_instances(img, prob_thresh=prob_thresh)
    
    print("Finished")
    return labels

In [83]:
mask0_3d = _segment3d(dat0_raw, sd_model, prob_thresh=None)

Finished


In [90]:
viewer = napari.view_labels(mask0_3d)
# viewer.add_image(dat0_raw)

In [ ]:
# sd_model.predict_instances()